# Lab 6: Word2Vec from scratch in TensorFlow

In this lab, we'll write Word2Vec, a particularly popular, simple, and powerful model for unsupervised learning of embedding vectors for words.

## Understanding Word2Vec
Word2Vec trains embeddings by using them as input to a logistic regression model, trained to predict a word given nearby words.

Alternatively, you can think of Word2Vec as a neural network with linear activations and one hidden layer, trained to predict a word given nearby words.
In this interpretation, the input and output of the model are both one-hot encoded vectors, and the embeddings of the words are the activations of the hidden layer given that word's one-hot encoding as input.
Since the inputs are one-hot encoded, it's more efficient (but equivalent) to use an embedding lookup instead of a matrix multiply for the first layer.

The data is produced by taking any large body of text (e.g. Wikipedia) and creating (context, target) pairs, where the target is any word and the context is the $n$ words to its left and right.

There are two common ways of training Word2Vec:
 - The "skip-gram" model uses the target word as input and predicts context words.
 - The "continuous bag-of-words" (CBOW) model uses the context words as input and predicts the target word
 
We'll focus on the CBOW model, since it tends to work better for small datasets.

The process of generating the data for CBOW is as follows:
 1. Tokenize (convert words, which are strings, to integer "tokens" indicating the word) the dataset by assigning each word a unique integer (integer encoding)
 2. For each word in the dataset, create a single (`context`, `target`) pair, where `target` is the integer encoding of the word and `context` is a list of the integer encodings of the $n=1$ words to the left and right of the target word

(I've already written the code to do this for you below)
 
Then, the model is trained to predict the one-hot encoding of the target word given the integer encodings of the context words:
 1. For each context word, look up its embedding in a table
 2. Combine these into an "average context embedding," which is the depth-wise average of the embeddings of the individual context words. This results in a single embedding vector, which acts as the "total context" in some sense.
 3. Perform a logistic regression (equivalently, a single dense layer with softmax activation) to predict the target word using the average context embedding as input.

Instead of full logistic regression, which uses the softmax function over all of the many words that appear in the dataset, we will use candidate sampling (specifically, noise-contrastive estimation) to compute the loss.
This should speed up training significantly.

For more info on Word2Vec, see [TensorFlow's "Vector Representations of Words" tutorial](https://www.tensorflow.org/tutorials/representation/word2vec), or Alex Minnaar's two tutorials, [one on the skip-gram model](http://alexminnaar.com/author/word2vec-tutorial-part-i-the-skip-gram-model.html) and [the other on the CBOW model](http://alexminnaar.com/author/word2vec-tutorial-part-ii-the-continuous-bag-of-words-model.html)

## Section 0: Download and preprocess the data
The dataset is the Cornell Movie-Dialogs Corpus, a collection of dialogue from movie scripts.
Download it from the link [here](http://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html), then unzip it in a subfolder called "data".
Your directory tree should include the file `./data/cornell movie-dialogs corpus/movie_lines.txt`.

I've written all the code for loading and preprocessing the data below, but read through it to understand what's going on.
You'll need to use parts of it later.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

Using TensorFlow backend.


### 0.1: Read the data
First, we read the lines as separate strings from the text file.

If you are running into memory problems (i.e. running out of RAM while working on this assignment) you can increase `skip_header` to train on a smaller dataset (but your resulting embeddings will be less good).
There are about 300,000 lines total, so you can set `skip_header` to skip some percentage of those.

In [2]:
# Ignore warnings on incorrectly-formatted inputs
import warnings
warnings.filterwarnings('ignore')

_, _, _, _, lines = np.genfromtxt(
    './data/cornell movie-dialogs corpus/movie_lines.txt',
    dtype='<U128', 
    delimiter='+++$+++', autostrip=True,
    encoding='latin1', invalid_raise=False, unpack=True,
    skip_header=0)

### 0.2: Tokenize
Keras has a built-in tokenization utility, which we'll use.
This creates two dictionaries, which you'll need to use later:
 - `tokenizer.word_index` maps from string words to integer tokens
 - `tokenizer.index_word` maps from integer tokens to string words

Then, we convert the dialogue lines into lists of tokens.

In [3]:
# Create a tokenizer and assign each word an integer
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(lines)

# Convert the lines to lists of integers
tokenized_lines = tokenizer.texts_to_sequences(lines)

# Delete the original lines to save memory
del(lines)

# Here's an example of how to use word_index and index_word
print('The integer encoding of "yes" is:',
      tokenizer.word_index['yes'])
print('The word with integer encoding 10 is:',
      tokenizer.index_word[10])

The integer encoding of "yes" is: 67
The word with integer encoding 10 is: what


### 0.3: Create (context, target) pairs
In this case, "target" means the integer encoding of a word, and "context" means a list of the integer encodings of the words to its left and right.

In [4]:
window_size = 1  # Consider 1 word on each side of target
stride = 2       # Avoid window overlap

X = []
Y = []

# Add (context, target) pairs to the dataset
for line in tokenized_lines:
    # Do not use lines that are too short
    if len(line) < 2 * window_size + 1:
        continue
    
    for target_idx in range(window_size, 
                            len(line) - window_size, 
                            stride):
        target = line[target_idx]
        left_context = line[target_idx - window_size : 
                            target_idx]
        right_context = line[target_idx + 1 :
                             target_idx + window_size + 1]
        
        Y.append(target)
        X.append(left_context + right_context)

# Convert to ndarrays
X = np.asarray(X)
Y = np.asarray(Y)

# These constants may be useful for you later
n_words = max(tokenizer.word_index.values())
n_examples = len(X)

print('n_words:', n_words)
print('n_examples:', n_examples)

n_words: 54034
n_examples: 1178349


### 0.4: Make a TSV metadata file
This creates a metadata file in the format that the TensorBoard Projector uses (tab-separated values).
This will let us see the names for words later when we visualize the embedding.

In [5]:
import os

# Make a logs directory if none exists yet
os.makedirs('./logs', exist_ok=True)

# Make TSV metadata file
with open('./logs/metadata.tsv', 'w') as f:
    # Header specifies column name
    f.write('Word\tIndex\tCount\n')
    
    # Unrecognized values have an integer encoding of 0
    f.write('UNK\t0\t0\n')
    
    # One word per line
    for i in range(1, n_words):
        word = tokenizer.index_word[i]
        count = tokenizer.word_counts[word]
        f.write('{}\t{}\t{}\n'.format(word, i, count))

### 0.5: Build a TensorFlow data pipeline
I've set up the `tf.data.Dataset` and the iterator for you.
Feel free to change the training hyperparameters and transforms if you like.

You might want to try:
 - Changing the batch size to suit your RAM / GPU memory situation
 - Removing `.cache()` from the transforms if you can't fit the whole dataset in RAM
 - Changing n_negative_samples. Increasing it makes computing the loss slower but more accurate per batch. 

In [6]:
# Training hyperparameters
n_epochs = 5
batch_size = 256
n_batches_per_epoch = n_examples / batch_size
print('Batches per epoch:', n_batches_per_epoch)

# Number of wrong words to sample when computing
# the loss using noise-contrastive estimation.
n_negative_samples = 128

# Construct dataset and apply transforms
dataset = tf.data.Dataset.from_tensor_slices((X, Y))\
    .shuffle(1000)\
    .batch(batch_size)\
    .cache()\
    .repeat(n_epochs)

# A one-shot iterator should be fine here
iterator = dataset.make_one_shot_iterator()

Batches per epoch: 4602.92578125


## Section 1: Build a model graph

### 1.0: Model hyperparameters
`embedding_size` is the number of dimensions in the embedding vector of each word.
Try to train 64-dimensional embeddings, but if training takes too long, feel free to reduce this to 32 or 16.

`validation_words` is a list of words we'll use to see how good our embeddings are.
While training, we'll periodically print out the words that have embeddings most similar to these words.
As training progresses, this should start printing words with similar meanings.

In [7]:
embedding_size = 64
validation_words = ['yes', 'small', 'thousand']

### 1.1: Input tensors
Using the iterator, get a tensor which holds the context integer encodings (shape: `(batch_size, 2)`) and a tensor which holds the target integer encoding (shape: `(batch_size,)`).

In [8]:
with tf.name_scope('inputs'):
    next_elem = iterator.get_next()
    context = tf.identity(next_elem[0], name='features')
    target = tf.identity(next_elem[1],name='digit')

### 1.2: Embedding variable
Make a variable which holds the embeddings for each of the words.
It should be a rank-2 tensor (a matrix) where the $i$-th row is the embedding vector for the word with integer encoding $i$.
Its shape should be `(n_words, embedding_size)`.

In [9]:
embeddings = tf.get_variable('embeddings', shape=(n_words, embedding_size), dtype=tf.float32,
                          initializer=tf.glorot_uniform_initializer())

Instructions for updating:
Colocations handled automatically by placer.


### 1.3: Compute the average context embedding
Look up the embeddings of each of the context words (with a single call to `tf.nn.embedding_lookup`), then average them depth-wise into a single average embedding vector with shape `(batch_size, embedding_size)`.

In [10]:
word_embeddings = tf.nn.embedding_lookup(embeddings, context,
                                         name='word_embeddings')

word_embeddings = tf.reduce_mean(word_embeddings,axis=1)

### 1.4: Create the variables for logistic regression
Create weight and bias variables for a logistic regression which takes in the average context embedding and outputs a probability for each word.

NOTE: We won't actually ever compute the output of this logistic regression by hand (i.e. the matrix multiplication and softmax activation), since we're using noise-contrastive estimation to approximate it.
TensorFlow's `nce_loss` function just takes the weights and bias tensors as input.

In [11]:
weights = tf.get_variable('weights', shape=(n_words, embedding_size), dtype=tf.float32,
                          initializer=tf.glorot_uniform_initializer())
bias = tf.get_variable('bias', shape=(n_words), dtype=tf.float32,
                       initializer=tf.zeros_initializer())

### 1.5: Compute the loss
We want to jointly train the logistic regression weights and the word embeddings using cross-entropy loss on the output of the logistic regression, but this is inefficient because of the large number of outputs (we'd need one logit for each word).

Instead, approximate the per-example loss using noise-contrastive estimation by calling `tf.nn.nce_loss`, then compute the mean loss for the batch, and add a summary scalar to plot the loss in TensorBoard.

In [24]:
loss_per_example = tf.nn.nce_loss(weights, bias, tf.expand_dims(target,-1), word_embeddings, n_negative_samples, n_words)a
loss_batch = tf.reduce_mean(loss_per_example)

tf.summary.scalar('loss', loss_batch)

(?,)


<tf.Tensor 'loss_1:0' shape=() dtype=string>

### 1.7: Optimizer and gradients
Make an optimizer (I used `tf.train.AdamOptimizer` with `learning_rate=1e-3`) and an operation to apply the gradient updates.

In [13]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
optimize_step = optimizer.minimize(loss_batch)

Instructions for updating:
Use tf.cast instead.


### 1.8: Operations find similar embeddings
The **cosine similarity** between two embeddings $\vec{x}$ and $\vec{y}$ is the cosine of the angle between them, computed as:
$$
\cos(\vec{x}, \vec{y}) = \frac{\vec{x} \cdot \vec{y}}{||\vec{x}|| \cdot ||\vec{y}||}
$$
This is a more robust similarity measure than Euclidean distance, since it's invariant to scaling the embedding vectors by a constant.

We'll use cosine similarity to find the most similar words to any given input word.
The steps for doing this are:
 1. Take a word's integer encoding as input, and compute its embedding (I've done this for you)
 2. Compute a "similarity tensor" which holds cosine similarity of this embedding with all of the word embeddings. This should result in a single tensor with shape `(n_words,).
 3. Use `tf.nn.top_k` to find the top 8 similarities and their indices in the similarity tensor. These indices will be the integer encodings of the words with the most similar embeddings to the input.

Later, you can find the most similar words to a given input word by running (in a session) the tensor that holds the top 8 indices, then using `tokenizer.index_word` on each of those indices.

When debugging this, make sure that the most similar word to a given word is that word itself, with a cosine similarity of 1.

In [14]:
with tf.name_scope('compute_similar_words'):
    input_word = tf.placeholder(tf.int64, shape=(), name='input_word')
    input_embedding = tf.nn.embedding_lookup(embeddings,
                                             input_word,
                                             name='input_embedding')
    
    similarity = tf.reduce_sum(tf.multiply(tf.nn.l2_normalize(input_embedding,0), tf.nn.l2_normalize(embeddings,1)), axis=1)
    top_8_vals, top_8_indices = tf.nn.top_k(similarity, 8)
    
    

## Section 2: Train the model
Use the same kind of training loop we've used before, repeatedly calling the operation that applies gradient updates.

The dataset is large, but the model doesn't do much work for each example.
So, you might want to only run and save the summaries every 1000 batches or so to speed up training.

In addition, every 1000 batches, for every word in `validation_words`, print the 8 most similar words by cosine similarity.

Finally, use a `tf.train.Saver()` to save the model in `./logs`, which is necessary to visualize the embeddings in TensorBoard.

In [27]:
saver = tf.train.Saver()
merged_summaries = tf.summary.merge_all()

train_batch = 0

with tf.Session() as sess:
    # Make one writer for training logs and another for test logs 
    train_writer = tf.summary.FileWriter('./logs/train', graph=sess.graph)
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    print(embeddings.eval()[0])  
    
    while True:
        try:
            if train_batch > 0: # Typical batch
                if train_batch % 1000 == 0:
                    summary, loss, _ = sess.run([merged_summaries, loss_batch, optimize_step])
                    train_writer.add_summary(summary, train_batch)
                    print('batch: ', train_batch)
                    print('loss:', loss)
                    for word in validation_words:
                        print('validation word: ', word)
                        id_num = tokenizer.word_index[word]
                        indices, vals = sess.run([top_8_indices, top_8_vals], feed_dict={input_word:id_num})
                        results = zip(indices, vals)
                        for i, v in results:
                            print('word: ', tokenizer.index_word[i], '        closeness: ', v)
                    print(embeddings.eval()[0])        

                # Run summary ops and optimization
                else:
                    sess.run(optimize_step)
                train_batch += 1
            else:
                # On the first batch, run a full trace
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()

                summary, loss, _ = sess.run([merged_summaries, loss_batch, optimize_step],
                                      options=run_options,
                                      run_metadata=run_metadata)

                train_writer.add_summary(summary, train_batch)    
                train_writer.add_run_metadata(run_metadata, 
                                              'first training batch')
                train_batch += 1
        except tf.errors.OutOfRangeError: # No more data
            break
        
    # Save model
    saver.save(sess, './logs/model.ckpt')    

[ 0.01009468 -0.00897332 -0.00374673 -0.00250185  0.000675   -0.00918134
  0.00208945  0.00929843  0.00538841  0.00841895  0.0049213   0.00969562
  0.00510812  0.00671782 -0.00677457  0.00703699 -0.00571075 -0.00981901
 -0.00963414  0.00781782 -0.00770693 -0.00402639 -0.00197614  0.00624583
  0.00675544 -0.00147443  0.00992838  0.00233917 -0.00998277  0.00602558
 -0.01037628 -0.00133684 -0.00186734 -0.00476529 -0.00911917  0.00586512
  0.00538576  0.00601687  0.0029445  -0.00372859  0.00251298 -0.00937644
  0.00527893 -0.00964292  0.00730952  0.00690612 -0.00138271 -0.00342133
  0.00425649  0.00332499 -0.00735577  0.00268836 -0.0088688   0.00850492
  0.00556495 -0.0065638  -0.00919588  0.00805417 -0.00583127 -0.00854167
 -0.00854005  0.00606232 -0.00518793 -0.00018326]
batch:  1000
loss: 266.82703
validation word:  yes
word:  yes         closeness:  0.9999999
word:  it         closeness:  0.99994195
word:  some         closeness:  0.99994016
word:  and         closeness:  0.9999376
wor

KeyboardInterrupt: 

## Section 3: Visualize the learned embeddings
Run TensorBoard pointed at `./logs` and look in the Projector tab, then use the "Load data" button and select the `./logs/metadata.tsv` file we created earlier to load word labels.

Try typing some words into the search bar and see which words come up as most similar.

## Section 4 (optional): Generate analogies
Try using embedding vector arithmetic to generate analogies.
To do this:
 - Load the saved model into a `tf.Session()` with the saver
 - Compute the embedding vector for several vectors using `feed_dict` and `tf.nn.embedding_lookup`
 - Do vector arithmetic on the computed embeddings
 - Find the most similar word embeddings by cosine similarity, then find the words that map to those embeddings

In [ ]:
# Your code here?